# European Weightlifting Championships ETL Pipeline (2019-2024)

**Author:** Senior Python Data Engineer  
**Objective:** Extract, Transform, and Load weightlifting championship data from Wikipedia into a unified, analysis-ready dataset.

---

## 1. Import Dependencies

In [13]:
# Standard library imports
import re
import warnings
from typing import Dict, List, Optional, Tuple

# Third-party imports
import pandas as pd
import plotly.graph_objects as go
import requests

# Configuration
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# Headers to mimic browser request and avoid 403 errors
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
}

## 2. Configuration & Constants

In [14]:
# Year-to-URL mapping for all championships
YEAR_URLS: Dict[int, Dict[str, any]] = {
    2019: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2019',
        'male_table_idx': 5,
        'female_table_idx': 6
    },
    2020: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2020',
        'male_table_idx': 2,
        'female_table_idx': 3
    },
    2021: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2021',
        'male_table_idx': 2,
        'female_table_idx': 3
    },
    2022: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2022',
        'male_table_idx': 2,
        'female_table_idx': 3
    },
    2023: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2023',
        'male_table_idx': 2,
        'female_table_idx': 3
    },
    2024: {
        'url': 'https://es.wikipedia.org/wiki/Campeonato_Europeo_de_Halterofilia_de_2024',
        'male_table_idx': 2,
        'female_table_idx': 3
    }
}

# Medal order for sorting
MEDAL_ORDER = {'Oro': 1, 'Plata': 2, 'Bronce': 3}

# Output file path
OUTPUT_FILE = 'processed_weightlifting_data_2019_2024.csv'

## 3. Data Extraction & Parsing Functions

In [15]:
def parse_athlete_data(cell_value: str) -> pd.Series:
    """
    Parse athlete data from Wikipedia table cell using regex patterns.
    
    Expected format: "Athlete Name Country 123 + 456 = 579"
    Where: 123 = Snatch, 456 = Clean&Jerk, 579 = Total
    
    Args:
        cell_value: Raw cell content from Wikipedia table
        
    Returns:
        pd.Series: [Nombre, Pais, Arrancada, DosTiempos, Total]
        Returns None values if parsing fails
    """
    # Handle missing/invalid data
    if pd.isna(cell_value):
        return pd.Series([None] * 5)
    
    try:
        # Clean invisible characters and footnotes
        cleaned = str(cell_value).replace('\u200b', '').split('[')[0].strip()
        
        # Regex pattern to extract three numbers (snatch + clean&jerk = total)
        # Pattern: captures "123 + 456 = 579" or "123 456 579"
        number_pattern = r'(\d+)\s*(?:\+|)\s*(\d+)\s*(?:=|)\s*(\d+)'
        numbers_match = re.search(number_pattern, cleaned)
        
        if not numbers_match:
            return pd.Series([None] * 5)
        
        snatch = int(numbers_match.group(1))
        clean_jerk = int(numbers_match.group(2))
        total = int(numbers_match.group(3))
        
        # Extract everything before the numbers
        text_before_numbers = cleaned[:numbers_match.start()].strip()
        tokens = text_before_numbers.split()
        
        if len(tokens) < 2:
            return pd.Series([None] * 5)
        
        # Country is typically the last token before numbers
        country = tokens[-1]
        
        # Athlete name is everything before the country
        athlete_name = ' '.join(tokens[:-1])
        
        return pd.Series([athlete_name, country, snatch, clean_jerk, total])
        
    except Exception as e:
        # Log the error but continue processing
        print(f"⚠️  Parsing error for value '{cell_value}': {str(e)}")
        return pd.Series([None] * 5)

In [16]:
def extract_medal_data(
    table: pd.DataFrame,
    medal_type: str,
    gender: str,
    year: int
) -> pd.DataFrame:
    """
    Extract and structure data for a specific medal type.
    
    Args:
        table: Raw DataFrame from Wikipedia
        medal_type: 'Oro', 'Plata', or 'Bronce'
        gender: 'M' or 'F'
        year: Championship year
        
    Returns:
        pd.DataFrame: Structured medal data
    """
    df = table[['Evento', medal_type]].copy()
    df['Medalla'] = medal_type
    
    # Parse athlete data
    df[['Nombre', 'Pais', 'Arrancada', 'DosTiempos', 'Total']] = \
        df[medal_type].apply(parse_athlete_data)
    
    # Add metadata
    df['Género'] = gender
    df['Año'] = year
    
    # Clean up
    df = df.drop(columns=[medal_type])
    
    return df

In [17]:
def process_championship_year(
    year: int,
    url: str,
    male_table_idx: int,
    female_table_idx: int
) -> pd.DataFrame:
    """
    Process all data for a single championship year.
    
    Args:
        year: Championship year
        url: Wikipedia URL for the championship
        male_table_idx: Index of male results table
        female_table_idx: Index of female results table
        
    Returns:
        pd.DataFrame: Combined and cleaned data for the year
    """
    print(f"\n{'='*60}")
    print(f"Processing Year: {year}")
    print(f"{'='*60}")
    
    try:
        # Fetch page content with headers to avoid 403 error
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        
        # Use StringIO to pass HTML content to pandas
        from io import StringIO
        html_content = StringIO(response.text)
        
        # Fetch all tables from Wikipedia
        tables = pd.read_html(html_content)
        print(f"✓ Found {len(tables)} tables in the page")
        
        # Extract male data
        male_table = tables[male_table_idx].copy()
        male_table.columns = ['Evento', 'Oro', 'Plata', 'Bronce']
        
        male_oro = extract_medal_data(male_table, 'Oro', 'M', year)
        male_plata = extract_medal_data(male_table, 'Plata', 'M', year)
        male_bronce = extract_medal_data(male_table, 'Bronce', 'M', year)
        
        # Extract female data
        female_table = tables[female_table_idx].copy()
        female_table.columns = ['Evento', 'Oro', 'Plata', 'Bronce']
        
        female_oro = extract_medal_data(female_table, 'Oro', 'F', year)
        female_plata = extract_medal_data(female_table, 'Plata', 'F', year)
        female_bronce = extract_medal_data(female_table, 'Bronce', 'F', year)
        
        # Combine all data
        df_year = pd.concat([
            male_oro, male_plata, male_bronce,
            female_oro, female_plata, female_bronce
        ], ignore_index=True)
        
        # Data quality checks and cleaning
        initial_rows = len(df_year)
        
        # Drop rows with missing critical data
        df_year = df_year.dropna(
            subset=['Nombre', 'Pais', 'Arrancada', 'DosTiempos', 'Total']
        )
        
        # Fix country name inconsistencies
        df_year['Pais'] = df_year['Pais'].replace({'Unido': 'Reino Unido'})
        df_year['Nombre'] = df_year.apply(
            lambda row: row['Nombre'].replace(' Reino', '') 
            if row['Pais'] == 'Reino Unido' else row['Nombre'],
            axis=1
        )
        
        # Extract category and date from event
        df_year['Categoria'] = df_year['Evento'].str.extract(r'(\+?\d+\s?kg)')
        df_year['Fecha'] = df_year['Evento'].str.extract(r'\((\d{2}\.\d{2})\)')
        
        # Sort by medal priority and event
        df_year['OrdenMedalla'] = df_year['Medalla'].map(MEDAL_ORDER)
        df_year = df_year.sort_values(
            by=['OrdenMedalla', 'Evento']
        ).drop(columns='OrdenMedalla')
        
        df_year = df_year.reset_index(drop=True)
        
        dropped_rows = initial_rows - len(df_year)
        print(f"✓ Processed {len(df_year)} records ({dropped_rows} invalid rows dropped)")
        print(f"  - Male athletes: {len(df_year[df_year['Género'] == 'M'])}")
        print(f"  - Female athletes: {len(df_year[df_year['Género'] == 'F'])}")
        
        return df_year
        
    except Exception as e:
        print(f"❌ Error processing year {year}: {str(e)}")
        raise

## 4. ETL Pipeline Execution

In [18]:
# Process all championship years
all_years_data: List[pd.DataFrame] = []

for year, config in YEAR_URLS.items():
    df_year = process_championship_year(
        year=year,
        url=config['url'],
        male_table_idx=config['male_table_idx'],
        female_table_idx=config['female_table_idx']
    )
    all_years_data.append(df_year)

# Combine all years into unified dataset
df_all_years = pd.concat(all_years_data, ignore_index=True)

# Final sort by year and medal priority
df_all_years['OrdenMedalla'] = df_all_years['Medalla'].map(MEDAL_ORDER)
df_all_years = df_all_years.sort_values(
    by=['Año', 'OrdenMedalla', 'Evento']
).drop(columns='OrdenMedalla')
df_all_years = df_all_years.reset_index(drop=True)

print(f"\n{'='*60}")
print(f"ETL Pipeline Complete")
print(f"{'='*60}")
print(f"Total records: {len(df_all_years)}")
print(f"Years covered: {sorted(df_all_years['Año'].unique())}")
print(f"Countries: {df_all_years['Pais'].nunique()}")
print(f"Athletes: {df_all_years['Nombre'].nunique()}")


Processing Year: 2019
✓ Found 10 tables in the page
✓ Processed 59 records (1 invalid rows dropped)
  - Male athletes: 30
  - Female athletes: 29

Processing Year: 2020
✓ Found 7 tables in the page
✓ Processed 60 records (0 invalid rows dropped)
  - Male athletes: 30
  - Female athletes: 30

Processing Year: 2021
✓ Found 7 tables in the page
✓ Processed 60 records (0 invalid rows dropped)
  - Male athletes: 30
  - Female athletes: 30

Processing Year: 2022
✓ Found 7 tables in the page
✓ Processed 60 records (0 invalid rows dropped)
  - Male athletes: 30
  - Female athletes: 30

Processing Year: 2023
✓ Found 7 tables in the page
✓ Processed 60 records (0 invalid rows dropped)
  - Male athletes: 30
  - Female athletes: 30

Processing Year: 2024
✓ Found 7 tables in the page
✓ Processed 56 records (4 invalid rows dropped)
  - Male athletes: 27
  - Female athletes: 29

ETL Pipeline Complete
Total records: 355
Years covered: [np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), n

In [19]:
# Preview the unified dataset
print("\n📊 Dataset Preview:")
display(df_all_years.head(20))

print("\n📊 Dataset Info:")
display(df_all_years.info())

print("\n📊 Summary Statistics:")
display(df_all_years[['Arrancada', 'DosTiempos', 'Total']].describe())


📊 Dataset Preview:


,Evento,Medalla,Nombre,Pais,Arrancada,DosTiempos,Total,Género,Año,Categoria,Fecha
0,+109 kg (13.04),Oro,Lasha Talajadze,Georgia,218.0,260.0,478.0,M,2019,+109 kg,13.04
1,+87 kg (13.04),Oro,Tatiana Kashirina,Rusia,146.0,185.0,331.0,F,2019,+87 kg,13.04
2,102 kg (13.04),Oro,Dmytro Chumak,Ucrania,175.0,216.0,391.0,M,2019,102 kg,13.04
3,109 kg (13.04),Oro,Simon Martirosian,Armenia,192.0,235.0,427.0,M,2019,109 kg,13.04
4,45 kg (06.04),Oro,Şaziye Erdoğan,Turquía,75.0,88.0,163.0,F,2019,45 kg,06.04
5,49 kg (06.04),Oro,Elena Andrieș,Rumania,87.0,103.0,190.0,F,2019,49 kg,06.04
6,55 kg (06.04),Oro,Mirco Scarantino,Italia,116.0,145.0,261.0,M,2019,55 kg,06.04
7,55 kg (07.04),Oro,Joanna Łochowska,Polonia,87.0,112.0,199.0,F,2019,55 kg,07.04
8,59 kg (08.04),Oro,Rebeka Koha,Letonia,101.0,120.0,221.0,F,2019,59 kg,08.04
9,61 kg (07.04),Oro,Henadz Laptseu,Bielorrusia,133.0,153.0,286.0,M,2019,61 kg,07.04



📊 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Evento      355 non-null    object 
 1   Medalla     355 non-null    object 
 2   Nombre      355 non-null    object 
 3   Pais        355 non-null    object 
 4   Arrancada   355 non-null    float64
 5   DosTiempos  355 non-null    float64
 6   Total       355 non-null    float64
 7   Género      355 non-null    object 
 8   Año         355 non-null    int64  
 9   Categoria   355 non-null    object 
 10  Fecha       355 non-null    object 
dtypes: float64(3), int64(1), object(7)
memory usage: 30.6+ KB


None


📊 Summary Statistics:


,Arrancada,DosTiempos,Total
count,355.000000,355.000000,355.000000
mean,128.202817,155.459155,283.949296
std,37.881521,44.906562,81.947501
min,65.000000,11.000000,150.000000
25%,99.000000,120.500000,221.000000
50%,114.000000,140.000000,253.000000
75%,161.000000,196.000000,363.500000
max,222.000000,263.000000,485.000000


## 5. Data Export

In [20]:
# Save unified dataset
df_all_years.to_csv(OUTPUT_FILE, index=False)
print(f"\n✅ Data saved to: {OUTPUT_FILE}")
print(f"   File size: {len(df_all_years)} rows × {len(df_all_years.columns)} columns")


✅ Data saved to: processed_weightlifting_data_2019_2024.csv
   File size: 355 rows × 11 columns


## 6. Data Analysis & Visualization

### 6.1 Top 10 Countries by Medal Count

In [21]:
# Calculate medal counts by country
medallas_por_pais = df_all_years.groupby(['Pais', 'Medalla']).size().unstack(fill_value=0)
medallas_por_pais['Total'] = medallas_por_pais.sum(axis=1)
medallas_por_pais = medallas_por_pais.sort_values(by='Total', ascending=False)

print("Top 10 Countries by Total Medals:")
display(medallas_por_pais.head(10))

Top 10 Countries by Total Medals:


Medalla,Bronce,Oro,Plata,Total
Pais,,,,
Armenia,21,16,15,52
Bulgaria,6,18,11,35
Ucrania,6,15,12,33
Turquía,14,10,9,33
Rusia,11,5,13,29
Georgia,5,8,15,28
Italia,3,9,10,22
Rumania,6,11,4,21
Reino Unido,4,7,3,14


In [22]:
# Visualization: Top 10 Countries
top_10 = medallas_por_pais.head(10).drop(columns='Total')

fig = go.Figure()

for medalla in top_10.columns:
    fig.add_trace(go.Bar(
        name=medalla,
        x=top_10.index,
        y=top_10[medalla],
        hovertemplate='País: %{x}<br>Medallas: %{y}<extra></extra>'
    ))

fig.update_layout(
    barmode='stack',
    title='Top 10 Countries by Medal Count (2019–2024)',
    xaxis_title='Country',
    yaxis_title='Medal Count',
    legend_title='Medal Type',
    height=500,
    width=900
)

fig.show()
fig.write_image('chart_medallas.png', width=900, height=500)

### 6.2 Gender Equity Analysis - Medal Distribution

In [23]:
# Calculate gender distribution of medals
medallas_genero = df_all_years.groupby(['Pais', 'Género']).size().unstack(fill_value=0)

# Rename columns for clarity
if 'F' in medallas_genero.columns and 'M' in medallas_genero.columns:
    medallas_genero.columns = ['Femenino', 'Masculino']

# Calculate equity (smaller difference = more equitable)
medallas_genero['Diferencia'] = abs(
    medallas_genero['Femenino'] - medallas_genero['Masculino']
)
medallas_equidad = medallas_genero.sort_values(by='Diferencia')

print("Top 10 Most Equitable Countries (by medal distribution):")
display(medallas_equidad.head(10))

Top 10 Most Equitable Countries (by medal distribution):


,Femenino,Masculino,Diferencia
Pais,,,
Irlanda,1,0,1
Alemania,4,3,1
Azerbaiyán,2,3,1
Serbia,1,0,1
Moldavia,4,5,1
Israel,1,0,1
Finlandia,1,0,1
Suecia,2,0,2
Austria,2,0,2


In [24]:
# Visualization: Gender Equity in Medals
top_equidad = medallas_equidad.head(10)[['Femenino', 'Masculino']]

fig = go.Figure()

for genero in top_equidad.columns:
    fig.add_trace(go.Bar(
        name=genero,
        x=top_equidad.index,
        y=top_equidad[genero],
        hovertemplate='País: %{x}<br>Medallas: %{y}<extra></extra>'
    ))

fig.update_layout(
    barmode='group',
    bargap=0.1,
    title='Top 10 Most Equitable Countries - Medal Distribution by Gender (2019–2024)',
    xaxis_title='Country',
    yaxis_title='Medal Count',
    legend_title='Gender',
    height=500,
    width=900
)

fig.show()
fig.write_image('chart_equidad_medallas.png', width=900, height=500)

### 6.3 Gender Equity Analysis - Performance Averages

In [25]:
# Calculate average total weight lifted by gender
promedios_total = df_all_years.groupby(['Pais', 'Género'])['Total'].mean().unstack()

# Rename columns
if 'F' in promedios_total.columns and 'M' in promedios_total.columns:
    promedios_total.columns = ['Femenino', 'Masculino']

# Calculate difference (gender performance gap)
promedios_total['Diferencia'] = abs(
    promedios_total['Femenino'] - promedios_total['Masculino']
)
promedios_equidad = promedios_total.sort_values(by='Diferencia')

print("Top 10 Countries with Most Similar Performance Between Genders:")
display(promedios_equidad.head(10))

Top 10 Countries with Most Similar Performance Between Genders:


,Femenino,Masculino,Diferencia
Pais,,,
Rumania,209.277778,247.000000,37.722222
Ucrania,227.666667,323.500000,95.833333
Alemania,222.750000,325.000000,102.250000
Turquía,198.666667,309.533333,110.866667
Francia,224.000000,341.000000,117.000000
Polonia,206.800000,324.000000,117.200000
Moldavia,239.500000,358.800000,119.300000
Georgia,248.000000,368.560000,120.560000
Letonia,221.666667,343.000000,121.333333


In [26]:
# Visualization: Performance Equity
top_promedios = promedios_equidad.head(10)[['Femenino', 'Masculino']]

fig = go.Figure()

for genero in top_promedios.columns:
    fig.add_trace(go.Bar(
        name=genero,
        x=top_promedios.index,
        y=top_promedios[genero],
        hovertemplate='País: %{x}<br>Promedio: %{y:.1f} kg<extra></extra>'
    ))

fig.update_layout(
    barmode='group',
    bargap=0.1,
    title='Top 10 Countries - Average Total Weight Lifted by Gender (2019–2024)',
    xaxis_title='Country',
    yaxis_title='Average Total (kg)',
    legend_title='Gender',
    height=500,
    width=900
)

fig.show()
fig.write_image('chart_equidad_total.png', width=900, height=500)